In [23]:
# Imports

import numpy as np

import torch
from torch import nn
from torch.autograd import Variable

In [24]:
# Constants
T = 5 # Iterations of RSGD
m = 1 # number of spherical
n = 1 # number of hyperbolic
V = 5 # Number of vertices in the graph
v = torch.rand(V, n+m+1)
J = torch.diag(torch.ones(V)) # Make identity matrix
J[0,0] = -1 # Convert the identity matrix to Minkowski Matrix

In [25]:
# Distance Functions

# Manually coded for a graph with a 5 vertices that makes a cycle
def graph_distance(i, j):
  if j-i == 1:
    return 1
  if j-i == 2:
    return 2
  if j-i == 3:
    return 2
  if j-i == 4:
    return 1
  return 0

In [26]:
# TODO
def spherical_distance(x, y):
  return torch.sqrt((y - x)**2)
# TODO
def hyperbolic_distance(x, y):
  return torch.sqrt((y - x)**2)

def euclidean_distance(x,y):
  return torch.sqrt((y - x)**2)

In [27]:
def product_space_distance(X, Y, m, n):

  total_distance = 0
  for i in range(m):
    total_distance += spherical_distance(X[i], Y[i])
  for i in range(m, m + n):
    total_distance += hyperbolic_distance(X[i], Y[i])
  for i in range(m + n, len(X)):
    total_distance += euclidean_distance(X[i], Y[i])

  return total_distance

In [28]:
# Loss Function

def embedding_loss(x_embedding, m, n):
  loss = torch.tensor(0.0)
  for i in range(V - 1):
    for j in range(i+1, V):
      ps_dist = product_space_distance(x_embedding[i], x_embedding[j], m, n)
      graph_dist = graph_distance(i,j)
      loss += (ps_dist/graph_dist - 1)**2

  return torch.sqrt(loss)

In [30]:
# Algorithm 1

x_t = torch.rand(V, n+m+1, requires_grad=True) # What should this be - (V, M+N+1) Tensor - embedding for every vertex in the graph
#print(x_t.shape)


for t in range(T): 
  loss = embedding_loss(x_t, m, n)
  print(loss.item())
  loss.backward() 
  h = x_t.grad
  #print(h.shape)
  for i in range(0, m):
    v[:,i] = h[:, i] - torch.dot(h[:, i], x_t[:, i]) * x_t[:, i]
  for i in range(m, m + n):
    v[:,i] = h[:, i] + torch.dot(torch.matmul(h[:, i].T, J), x_t[:, i]) * x_t[:, i] # Minkowski inner product
    v[:,i] = torch.matmul(J, v[:,i])
  # Euclidean
  v[:,m+n] = h[:, m +n]
  # Exponential Map - TODO
  for i in range(m + n + 1):
    with torch.no_grad():
      x_t[:,i] = torch.exp(v[:,i]) # Eponential map - How to do this correctly


1.5751817226409912
3.594003200531006
994093248.0
nan
nan


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2981.)
  


Scratch

In [21]:
x = torch.rand(3, 7)

x

tensor([[0.3663, 0.4816, 0.5139, 0.5029, 0.8967, 0.5575, 0.4727],
        [0.1491, 0.0482, 0.5064, 0.2531, 0.1705, 0.9138, 0.4764],
        [0.0981, 0.7895, 0.6832, 0.5813, 0.2268, 0.2185, 0.2900]])

In [22]:
x[:,0]

tensor([0.3663, 0.1491, 0.0981])